In [25]:
!pip install vaderSentiment

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 126.0/126.0 kB 3.7 MB/s eta 0:00:00


In [28]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 12.2/12.2 MB 11.5 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 11.7 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 12.7 MB/s eta 0:00:00
     ---------------------------------------- 96.8/96.8 kB ? eta 0:00:00
     ------------------------------------- 482.8/482.8 kB 10.3 MB/s eta 0:00:00
     ---------------------------------------- 48.9/48.9 kB ? eta 0:00:00
     -------------------------------------- 181.6/181.6 kB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 45.9/45.9 kB ? eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 13.6 MB/s eta 0:00:00
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [30]:
!pip install turkishnlp

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for turkishnlp: filename=turkishnlp-0.0.61-py3-none-any.whl size=7533 sha256=4374b261f70f9277b0a91889a488b6d611e7e0f3181cd67a27f2dfd3b60d1c45
  Stored in directory: c:\users\kdrcn\appdata\local\pip\cache\wheels\40\84\2b\2b0bcb710d42e3080099e6723f6e536832013fdfb5f23bb2fd
Successfully built turkishnlp


In [44]:
import pandas as pd
import re
import os
import sys
import spacy

import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.sentiment import SentimentIntensityAnalyzer

from textblob import TextBlob
from textblob import Word

from vaderSentiment import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

---

In [2]:
sw = set(stopwords.words("turkish"))

In [3]:
pd.set_option("display.max_colwidth", -1)

C:\Users\kdrcn\AppData\Local\Temp\ipykernel_8200\2475433900.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [4]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kdrcn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kdrcn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kdrcn\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kdrcn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
path = "128.turkish/restaurants_train_turkish.xml.dat"

---

In [6]:
with open(path, "r", encoding="utf-8") as file:
    text = file.read()
    file.close()
data = pd.DataFrame({"Text": text.split("\n"), "Tag": "Text"})

In [7]:
sentiment_index = data[data["Text"].isin(["Positive", "Negative", "Neutral"])].index
data.loc[sentiment_index, "Tag"] = "Sentiment"
data.loc[sentiment_index-1, "Tag"] = "Aspect"
df = pd.DataFrame({"Phrase": data[data["Tag"] == "Text"]["Text"].reset_index(drop=True), 
                   "Aspect": data[data["Tag"] == "Aspect"]["Text"].reset_index(drop=True), 
                   "Sentiment": data[data["Tag"] == "Sentiment"]["Text"].reset_index(drop=True)})
df = df.drop(1385, axis=0)

In [ ]:
df.head()

---

In [8]:
train = df.copy()

In [9]:
for index, row in train.iterrows():
    words = TextBlob(row["Phrase"]).words
    aspect = words.index("T")
    words[aspect] = row["Aspect"]
    row["Phrase"] = " ".join(words)

In [11]:
train["Phrase"] = train["Phrase"].apply(lambda x: " ".join(x.lower() for x in x.split()))
train["Aspect"] = train["Aspect"].apply(lambda x: " ".join(x.lower() for x in x.split()))

train["Phrase"] = train["Phrase"].apply(lambda x: " ".join(re.sub("[^\w\s]", "", x) for x in x.split()))
train["Aspect"] = train["Aspect"].apply(lambda x: " ".join(re.sub("[^\w\s]", "", x) for x in x.split()))

train["Phrase"] = train["Phrase"].apply(lambda x: " ".join(re.sub("\d", "", x) for x in x.split()))
train["Aspect"] = train["Aspect"].apply(lambda x: " ".join(re.sub("\d", "", x) for x in x.split()))

train["Phrase"] = train["Phrase"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

# train["Phrase"] = train["Phrase"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [39]:
train.head(10)

,Phrase,Aspect,Sentiment
0,manzara sahane evet servis rezalet,manzara,Positive
1,manzara sahane evet servis rezalet,servis,Negative
2,soguk su isteyince soguk yok butun sulari disari cikardik diyen garson siparisten tam saat sonra gelen yemekler yemegin yaninda soylenen roka pureden eser olmamasi istedigimiz mezenin kalmamis olmasi,mezenin,Negative
3,soguk su isteyince soguk yok butun sulari disari cikardik diyen garson siparisten tam saat sonra gelen yemekler yemegin yaninda soylenen roka pureden eser olmamasi istedigimiz mezenin kalmamis olmasi,garson,Negative
4,yemekler iyi hos lezzetler iyi heyecan verici bi taraflari yok iyi bir baligi iyi bir sekilde izgara yapmak artik atla deve bi olay degil,yemekler,Positive
5,yemekler iyi hos lezzetler iyi heyecan verici bi taraflari yok iyi bir baligi iyi bir sekilde izgara yapmak artik atla deve bi olay degil,lezzetler,Positive
6,servise fiyatlar cok fazla,fiyatlar,Negative
7,lacivert beni kalitesinin dusukluguyle cok sasirtti,kalitesinin,Negative
8,buraya kadar iyi yorum yapanlar halde önce balik yemediler,balik,Negative
9,kalamardan koladan önce balik geldi apar topar önümüze atildi,balik geldi,Negative
